In [9]:
import numpy as np
from scipy.sparse import  issparse

In [10]:
def detect_dependencies_via_rank(A, feasibility_tolerance=1e-8):
    """
    Detects dependent rows by comparing the rank of the matrix with and without each row.

    Parameters:
    - A: numpy.ndarray, the matrix to analyze.
    - feasibility_tolerance: float, threshold for rank determination.

    Returns:
    - dependent_rows: list of row indices that are linearly dependent.
    """
    from numpy.linalg import matrix_rank

    dependent_rows = []
    full_rank = matrix_rank(A, tol=feasibility_tolerance)

    for i in range(A.shape[0]):
        A_reduced = np.delete(A, i, axis=0)
        reduced_rank = matrix_rank(A_reduced, tol=feasibility_tolerance)

        if reduced_rank == full_rank:
            dependent_rows.append(i)

    return dependent_rows


In [16]:
A = np.array([[1, 2, 3],
                [2, 4, 6],
                [7, 8, 9],
                [10, 11, 13]])


In [12]:
detect_dependencies_via_rank(A)

[0, 1, 2]

In [13]:

def linear_dependency(A, b, feasibility_tolerance=0.01):
    """
    This function checks for linear dependency among the rows of matrix A.

    Linear dependency is checked through division of corresponding row elements.
    If the division of any corresponding row elements across two rows has very little variation,
    it indicates a row is linearly dependent on the other.

    Parameters:
    - A: numpy.ndarray or scipy sparse matrix, matrix of coefficients of the linear constraints.
    - feasibility_tolerance: float, the tolerance limit under which two constrained row values
                             are considered the same (indicating low variety, hence, dependence).

    Returns:
    - vector_index: list of tuple, the recorded pairs of indices of rows that were compared
                    and found to have equal implications of each other's space.
    - any_dependency: bool, True if there is at least one set of rows in A that are
                      linearly dependent within the threshold; False otherwise.
    """

    # Convert sparse matrix to dense if necessary.
    if issparse(A):
        A = A.toarray()

    # Convert b to a column vector
    b_column = np.array(b).reshape((-1, 1))

    # Stack A and b_column horizontally
    A = np.hstack((A, b_column))

    m, n = A.shape
    dependent_rows = [[] for _ in range(m)]  # Initialize with empty lists
    has_linear_dependency = np.zeros(m, dtype=bool)
    A_bool = A != 0
    for i in range(m):
        for j in range(i + 1, m):
            with np.errstate(divide='ignore', invalid='ignore'):
                same_non_zero = np.array_equal(A_bool[i], A_bool[j])
                if same_non_zero:
                    is_nonzero = (A[j, :] != 0)
                    div = np.where(is_nonzero, A[i, :] / A[j, :], np.inf)
                    div_filtered = div[np.isfinite(div)]

                    if len(div_filtered) > 0:
                        # Choose an element with a real value for comparison
                        div_with_value = div_filtered[0]
                        # Check if the differences are within the tolerance
                        close_enough = np.all(np.abs(div_filtered - div_with_value) < feasibility_tolerance)
                        if close_enough:
                            has_linear_dependency[i] = True
                            has_linear_dependency[j] = True
                            dependent_rows[i].append(j)
                            dependent_rows[j].append(i)

    return dependent_rows, has_linear_dependency


In [27]:
import numpy as np

def gauss_jordan_elimination_with_pivoting(A, tol=1e-9):
    """
    Performs Gauss-Jordan elimination with partial pivoting on matrix A
    to identify linearly dependent constraints.

    Parameters:
    A (numpy.ndarray): The input matrix (m x n).
    tol (float): Tolerance for detecting negligible elements.

    Returns:
    numpy.ndarray: The matrix in reduced row-echelon form.
    list: Indices of linearly dependent rows.
    """
    A = A.astype(float)  # Ensure the matrix is of type float for division
    m, n = A.shape
    pivot_row = 0
    pivot_cols = []
    dependent_rows = []
    
    for pivot_col in range(n):
        # Partial pivoting: Find the row with the maximum element in the current column
        max_row = np.argmax(np.abs(A[pivot_row:, pivot_col])) + pivot_row
        max_elem = np.abs(A[max_row, pivot_col])
        if max_elem < tol:
            # If the maximum element is negligible, skip this column
            continue
        # Swap the current row with the row having the maximum element
        if max_row != pivot_row:
            A[[pivot_row, max_row]] = A[[max_row, pivot_row]]
        # Normalize the pivot row
        A[pivot_row] = A[pivot_row] / A[pivot_row, pivot_col]
        # Eliminate the entries in the pivot column for all other rows
        for r in range(m):
            if r != pivot_row:
                A[r] = A[r] - A[r, pivot_col] * A[pivot_row]
        pivot_cols.append(pivot_col)
        pivot_row += 1
        if pivot_row == m:
            break
    # Identify dependent rows (rows that are all zeros after elimination)
    for i in range(pivot_row, m):
        if np.all(np.abs(A[i]) < tol):
            dependent_rows.append(i)
    return A, dependent_rows

# Example usage:
if __name__ == "__main__":
    # Define a matrix with linearly dependent rows
    A = np.array([
        [1, 2, 3],
        [4, 5, 6.000000000001],  # This row is a multiple of the first row
        [7, 8, 9]
    ])
    rref_matrix, dependent_rows = gauss_jordan_elimination_with_pivoting(A)
    print("Reduced Row-Echelon Form:\n", rref_matrix)
    print("Linearly Dependent Rows:", dependent_rows)


Reduced Row-Echelon Form:
 [[ 1.00000000e+00  0.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  2.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  9.99311744e-13]]
Linearly Dependent Rows: [2]


In [20]:
linear_dependency(A, [2, 4, 6.00001, 10])

([[1], [0], [], []], array([ True,  True, False, False]))